## קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## פונקציות

In [3]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [4]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [5]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [6]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [7]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [8]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


## code

### עיבוד חומר מקדים

In [9]:
v_date='230720'

In [10]:
taz_2020_catalog=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files','forecast_2020_{}_age_dis_type'.format(v_date))

taz_2020_aprt=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output','2020_jtmt_forcast_full_{}'.format(v_date))

In [11]:
v_date='230720'

In [12]:
save_file=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\bau\jew_kibolet\output'

kibolt_jew_2050=up_load_df(save_file,'{}_kibolt_jew_till_2050'.format(v_date)).set_index('Taz_num')

In [13]:
age_catalog=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files','age_des_types')

taz_with_aprt_diversity=up_load_shp(r'W:\Data\Forecast\creat_age_distribution\v_1\backround_files\shp\cost_diversity_for_age_distribution.shp').set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.merge(taz_2020_aprt[['Taz_num','zonetype','aprt_20','pop_without_dorms_yeshiva','Taz_name','Muni_Heb']],on='Taz_num',how='left').set_index('Taz_num')

taz_2020_catalog['taz_with_aprt_diversity']=taz_with_aprt_diversity['div']

taz_2020_catalog['taz_expensive']=taz_with_aprt_diversity['cost']

col=['classification_name','age_cycle']

taz_2020_catalog=taz_2020_catalog.reset_index().merge(age_catalog[col],on='classification_name',how='left').set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.join(kibolt_jew_2050).query('sector_for_age != "Arab"')

taz_2020_catalog=taz_2020_catalog.fillna(0)

In [14]:
folder_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תרחישי_מלל\מרכיבי_תחזית'

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

lst_change_sector=list(taz_sector_change.Taz_num)


taz_2020_catalog.loc[(taz_2020_catalog.index.isin(lst_change_sector)),'sector_for_age']='U_Orthodox'

## חישוב 

In [15]:
singal_year=list(range(2025,2051,5))

In [16]:
years=['add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

y_before=['2020','2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045']

only_years=['2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045',
 '2045_2050']

In [17]:
taz_2020_catalog['age_cycle_time']=0
taz_2020_catalog['current_aprt']=taz_2020_catalog['aprt_20']
years_to_change_cycle_stage=20
age_cycle_limit=4
taz_2020_catalog['age_cycle_2020']=taz_2020_catalog['age_cycle']
taz_2020_catalog['changed_this_cycle']=0

In [18]:
for y, y1 in zip(years, only_years):
    
        
    
    #### אזורים ללא יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים הראשוני

    taz_with_aprt_diversity_condition = taz_2020_catalog['taz_with_aprt_diversity']==1
    taz_expensive_condition = taz_2020_catalog['taz_expensive']==1
    

    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,' | new diverse neighborhood | ',1]

    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,' | new non-diverse expensive neighborhood  | ',1]
    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,' | new non-diverse neighborhood  | ',1]

    #### אזורים עם יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים כפונצקיה של יחס קיבולת למצב הקיים

    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']>0),'precent_add_to_current'
    ]=taz_2020_catalog['{}'.format(y)]/taz_2020_catalog['current_aprt']


    
    
    
    #### יחס נמוך - הקיבולת בטלה ביחס למצב הקיים
        #### יחס גדול - הקיבולת מבטלת את המצב הקיים

    kibolet_condition = taz_2020_catalog['precent_add_to_current'] > 0.5
    yush_condition =  taz_2020_catalog['zonetype']=='Judea and Samaria'

    taz_2020_catalog.loc[
        kibolet_condition &
        ~yush_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,'| renew diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        kibolet_condition &
        taz_expensive_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,'| a lot of new aprts in non-diverse expensive neighborhood|',1]

    taz_2020_catalog.loc[
        kibolet_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition &
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,'| a lot of new aprts in non-diverse neighborhood|',1]

    #### יחס בינוני

    ##### ביוש אין התחדשות עירונית לכן אם יחס גבוהה יחסית זה כמו בנייה חדשה עם אכלוס של סוג התפלגות גילים הראשוני

    
    kibolet_condition =  taz_2020_catalog['precent_add_to_current']>= 0.20

    taz_2020_catalog.loc[
        (yush_condition)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        (kibolet_condition) , ['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ] = [1, 0,'| a lot of new aprts in non-diverse neighborhood in yush |',1]


    ##### יוש אין התחדשות עירונית לכן אם יחס לא גבוהה יחסית זה מונע התבגרות של המצב הקיים
    

    ##### אם לא ביוש אז יש אפשרות התחדשות עירונית לכן המקום נהיה מגוון

    
    kibolet_condition =  (taz_2020_catalog['precent_add_to_current']>= 0.25) & (taz_2020_catalog['precent_add_to_current'] <= 0.5)
    

    taz_2020_catalog.loc[
        ~(yush_condition)&
        (kibolet_condition)&
        (taz_with_aprt_diversity_condition) , ['age_cycle','explanation_{}'.format(y1)]
    ] =  [3,'| a lot of new aprts in diverse neighborhood|']
    
    
    
    #השעון התקדם כבר ולא צריך לשנות כלום

    #### קוד לתהליך התבגרות התפלגות הגילים כפונקציה של סוג מגזר

    

    age_cycle_condition = taz_2020_catalog['age_cycle']<age_cycle_limit

    age_cycle_time_condition=(taz_2020_catalog['age_cycle_time']%years_to_change_cycle_stage==0)&(taz_2020_catalog['age_cycle_time']>0)

    sector_condition=taz_2020_catalog['sector_for_age']=='hitnachlut' 
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (age_cycle_condition),['explanation_{}'.format(y1),'changed_this_cycle']
    ]=['| natural aging |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle'
    ]=taz_2020_catalog['age_cycle']+1
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1


    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (sector_condition) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) & 
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5


    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==4

    

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
   
    
    sector_condition=taz_2020_catalog['sector_for_age']=='U_Orthodox'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1


    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
         (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[4,'| natural aging |',1]
    
    age_cycle_condition = taz_2020_catalog['age_cycle']==4
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    
   

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    taz_2020_catalog.loc[(taz_2020_catalog['changed_this_cycle']==0),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    taz_2020_catalog['changed_this_cycle']=0

    
    taz_2020_catalog['current_aprt']=taz_2020_catalog['current_aprt']+taz_2020_catalog['{}'.format(y)]
    taz_2020_catalog['current_aprt_{}'.format(y1)]= taz_2020_catalog['current_aprt']
    taz_2020_catalog['age_cycle_{}'.format(y1)]= taz_2020_catalog['age_cycle']
    taz_2020_catalog['age_cycle_time_{}'.format(y1)]= taz_2020_catalog['age_cycle_time']
    taz_2020_catalog['precent_add_to_current_{}'.format(y1)]= taz_2020_catalog['precent_add_to_current']

In [19]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

## ייצוא

In [20]:
taz_2020_catalog['hh_size_2020']=taz_2020_catalog['pop_without_dorms_yeshiva']/taz_2020_catalog['aprt_20']

In [21]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y_b)]
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y1)]+0.15
    

In [22]:
col_age=[ 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [23]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    
    col_taz=['Taz_num','current_aprt_{}'.format(y1),'explanation_{}'.format(y1),'age_cycle_{}'.format(y1),'hh_size_{}'.format(y1),'sector_for_age']
    
    left_col=['sector_for_age','age_cycle_{}'.format(y1)]
    right_col=['sector','age_cycle']

    df=taz_2020_catalog.reset_index()[col_taz].merge(age_catalog,left_on=left_col,right_on=right_col,how='left').set_index('Taz_num')
    

    df['pop']=df['current_aprt_{}'.format(y1)]*df['hh_size_{}'.format(y1)]
    df.loc[df['hh_size_{}'.format(y1)].isna(),'pop']=df['current_aprt_{}'.format(y1)]*df['hh_size']
    
        
    df['pop_{}'.format(sy)]=df['pop']
    
    df['aprt_{}'.format(sy)]=df['current_aprt_{}'.format(y1)]
    
    df[col_age]=df[col_age].multiply(df['pop'], axis="index")    
    
    file_save=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\bau\jew_pop'
    save_excel_path=r'{}\{}_forecast_pop_jewish_{}_bau.xlsx'.format(file_save,file_date,y1)
    
    df.to_excel(save_excel_path,index=True)
    
    taz_2020_catalog=taz_2020_catalog.join(df[['pop_{}'.format(sy),'aprt_{}'.format(sy)]])
    
 